In [1]:
from kipoiseq.extractors.protein import TranscriptSeqExtractor, CDSFetcher

from kipoiseq.transforms.functional import rc_dna, translate



In [2]:
from pathlib import Path
ddir = Path('/s/genomes/human/hg38/ensembl_GRCh38')
gtf_file = ddir / 'Homo_sapiens.GRCh38.99.gtf'
#gtf_file = 'tests/data/38_sample.gtf.txt'
protein_file = ddir / 'Homo_sapiens.GRCh38.pep.all.fa'

fasta_file = ddir / 'Homo_sapiens.GRCh38.dna.alt.fa'

fasta_file = ddir / 'Homo_sapiens.GRCh38.dna.primary_assembly.fa'
gps = TranscriptSeqExtractor(gtf_file, fasta_file)



In [3]:
from pyfaidx import Fasta
from tqdm import tqdm_notebook as tqdm
flag = False
def read_pep_fa(protein_file):
    import pandas as pd
    proteins = Fasta(str(protein_file))
    pl = []
    for v in proteins:
        names = v.long_name.split(" ", 8)
        d = {"protein_id": names[0], 'protein_type': names[1]}
        d = {**d, **dict([n.split(":", 1) for n in names[2:]])}
        d['seq'] = str(proteins[v.name])
        pl.append(d)
    return pd.DataFrame(pl)

dfp = read_pep_fa(protein_file)
dfp['transcript_id'] = dfp.transcript.str.split(".", n=1, expand=True)[0]
#assert not dfp['transcript_id'].duplicated().any()
dfp = dfp.set_index("transcript_id")
#dfp = dfp[~dfp.chromosome.isnull()]
assert len(gps) >100
assert gps.transcripts.isin(dfp.index).all()

'MRASPCISQPAASWHPRPSALRPTAGSGPDTRTPGTVEDGSAPCPAFRSPAVSPCGEEPCCFQISPAEETLELGRLVSPGCHKSSAPQERRLDPRARCVVHI'

In [4]:
div3_error = 0
seq_mismatch_err = 0
err_transcripts = []
for transcript_id in tqdm(gps.transcripts):
    # make sure all ids can be found in the proteome
    dna_seq = gps.get_seq(transcript_id)
    if dna_seq == "XXX":
        print(f"{transcript_id} has an ambiguous start and end.Skip!")
        continue
    # dna_seq = dna_seq[:(len(dna_seq) // 3) * 3]
    #if len(dna_seq) % 3 != 0:
     #   div3_error += 1
      #  print("len(dna_seq) % 3 != 0: {}".format(transcript_id))
       # err_transcripts.append({"transcript_id": transcript_id, "div3_err": True})
        #continue
    if len(dna_seq)%3 != 0:
        print(transcript_id)
        continue
    prot_seq = translate(dna_seq,hg38=True)
    if dfp.loc[transcript_id].seq != prot_seq:
        seq_mismatch_err += 1
        print("seq.mismatch: {}".format(transcript_id))
        n_mismatch = 0
        for i in range(len(prot_seq)):
            a = dfp.loc[transcript_id].seq[i]
            b = prot_seq[i]
            if a != b:
                n_mismatch += 1
                print("{} {} {}/{}".format(a,b,i,len(prot_seq)))
        err_transcripts.append({"transcript_id": transcript_id, "div3_err": False,
                            "n-seq-mismatch": n_mismatch})
        # print("prot:", dfp.loc[transcript_id].seq)
        # print("seq: ", prot_seq)
err_transcripts = pd.DataFrame(err_transcripts)

seq.mismatch: ENST00000640867
M I 0/212
Ambiguous start and end!
ENST00000606505 has an ambiguous start and end.Skip!
seq.mismatch: ENST00000639906
M L 0/235
Ambiguous start and end!
ENST00000473363 has an ambiguous start and end.Skip!
Ambiguous start and end!
ENST00000485079 has an ambiguous start and end.Skip!
seq.mismatch: ENST00000639044
M L 0/235
No tags for ambiguous start and end, but len % 3 != 0
ENST00000434431 has an ambiguous start and end.Skip!
seq.mismatch: ENST00000640958
M I 0/713
Ambiguous start and end!
ENST00000490594 has an ambiguous start and end.Skip!
seq.mismatch: ENST00000638727
M L 0/397
seq.mismatch: ENST00000640105
M L 0/434
seq.mismatch: ENST00000640797
M L 0/607
seq.mismatch: ENST00000640433
M L 0/643
Ambiguous start and end!
ENST00000418776 has an ambiguous start and end.Skip!
seq.mismatch: ENST00000639474
M L 0/258
seq.mismatch: ENST00000639640
M L 0/746
seq.mismatch: ENST00000639195
M L 0/754
seq.mismatch: ENST00000614010
M L 0/288
seq.mismatch: ENST00000

In [ ]:
dna_seq = gps.get_seq("ENST00000467125")
print((dna_seq[:]))

In [ ]:
k = "dobre"
print(k[-3:])
print(k[:-3])

In [ ]:
len(gps)

In [ ]:
len(dfp)